**This notebook creates the image patches**

In [18]:
# add ASAP path to sys to locate the multiresolutionimageinterface
import sys
sys.path.append('/opt/ASAP/bin')
# required libraries
import multiresolutionimageinterface as mir
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook

In [38]:
# Image size
sz = 256
# Levels to sample
levels = [0,2,3]

In [3]:
# directory to save patches
dirPatches = 'data/patches/'

df_train = pd.read_csv('data/training/training.csv').set_index('Unnamed: 0')
df_val_1 = pd.read_csv('data/training/validation_1.csv').set_index('Unnamed: 0')
df_val_2 = pd.read_csv('data/training/validation_2.csv').set_index('Unnamed: 0')
df_test = pd.read_csv('data/training/test.csv').set_index('Unnamed: 0')

# we make image patches from all of these so df's can be combined
df = pd.concat([df_train, df_val_1, df_val_2, df_test], ignore_index=True)
len(df)

143896

When making pathes, it is faster to process all patches from one WSI at the time, so we should loop over each WSI.

In [4]:
dirData = 'data/training/'
ImageFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirData):
    for file in f:
        if '.tif' in file and 'mask' not in file:
            ImageFiles.append(os.path.join(r, file))

In [25]:
def getSamples(mr_image, x_cent, y_cent, levels=[0,3]):
    channels = 3
    imgs = np.zeros((len(levels), sz, sz, channels), dtype=np.uint8)
    for i, lev in enumerate(levels):
        ds = mr_image.getLevelDownsample(lev)
        imgs[i] = mr_image.getUCharPatch(int(x_cent - (ds*sz/2)),
                                         int(y_cent - (ds*sz/2)),
                                         sz,
                                         sz,
                                         lev)
    return imgs

In [ ]:
for WSI in tqdm_notebook(ImageFiles, "Patches from WSI's"):
    # extract patient and node from the WSI path
    split = WSI.split('/')
    cnt = int(split[2].strip('center_'))
    splitpatient = split[3].split('_')
    patient = int(splitpatient[1])
    node = int(splitpatient[3].strip('.tif'))
    
    # then, subsample the df to get only pathes from this WSI
    df_sub = df[(df.patient == patient) & (df.node == node)]
    
    # load WSI image
    reader = mir.MultiResolutionImageReader()
    mr_image = reader.open(WSI)
    
    for i in range(len(df_sub)):
        id = str(df_sub.iloc[i].patchId)
        fileNamePrefix = dirPatches + id
        
        center_x = df_sub.iloc[i].centerX
        center_y = df_sub.iloc[i].centerY
        
        # note the flipped order of x and y
        imgs = getSamples(mr_image, center_y, center_x, levels)
        for lev_index, img in enumerate(imgs):
            fileName = fileNamePrefix + '_' + str(levels[lev_index]) + '.png'
            cv2.imwrite(fileName, img)

In [37]:
test_image = cv2.imread('data/patches/1066240043904_0.png')
plt.imshow(test_image)